In [32]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv

from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

In [33]:
load_dotenv('.env')
os.environ['OPENAI_MODEL_NAME'] = "gpt-4o-mini"
API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
client = OpenAI(api_key=API_KEY)

In [ ]:
topic = input("Enter the audit topic: ")
if topic:
    print("You entered:", topic)
else:
    print("Please enter an audit task.")

In [ ]:
tool = SerperDevTool(n_results=10, country="sg")

agent = Agent(
    role="search agent",
    goal="return a list of websites that contain most relevant information, in the context of Singapore, that is related to policy and regulation on {topic} only." ,
    backstory="""You're working on genereating a list of websites that contain most relavant policy and regualtion information on {topic} only..
    """,
    allow_delegation=False,
    verbose=True,
)

task = Task(
    description="""return a list of websites that contain most relevant information, in the context of Singapore, that is related to policy and regulation on {topic} only
    """,
    expected_output="""\
    A list of websites that contain most relavant policy and regualtion information on {topic} only.""",

    agent=agent,
    tools=[tool],
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True
)

result = crew.kickoff(inputs={"topic": topic})

In [ ]:
print(f"Raw Output: {result.raw}")
print(f"Token Usage: {result.token_usage}")
print("-----------------------------------------\n\n")
print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
print("-----------------------------------------\n\n")

In [ ]:
# tool = SerperDevTool(n_results=3, country="sg")

# agent = Agent(
#     role="search agent",
#     goal="return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}" ,
#     backstory="""You're working on gathering all relavant policy and regualtion information on {topic}.
#     """,
#     allow_delegation=False,
#     verbose=True,
#     max_iter=40
# )

# task = Task(
#     description="""return most relevant information, in the context of Singapore, that is related to policy and regulation on {topic}
#     """,
#     expected_output="""\
#     A detailed report with most relevant information information that is related to policy and regulation on {topic}.""",

#     agent=agent,
#     tools=[tool],
# )

# crew = Crew(
#     agents=[agent],
#     tasks=[task],
#     verbose=True
# )

# result = crew.kickoff(inputs={"topic": topic})

In [ ]:
# print(f"Raw Output: {result.raw}")
# print(f"Token Usage: {result.token_usage}")
# print("-----------------------------------------\n\n")
# print(f"Tasks Output of Task 1: {result.tasks_output[0]}")
# print("-----------------------------------------\n\n")
# # print(f"Tasks Output of Task 2: {result.tasks_output[1]}")